In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from pynndescent import NNDescent
import os
import xnetwork as xn
import scipy.sparse
import numpy as np
import igraph as ig


In [51]:
data_path='/Users/fsfatemi/local_AD/KBASE_datashare'
data=pd.read_csv(data_path+'/residuals_cqn_cpm.13603_1140.csv')
data.set_index(data.columns[0], inplace=True)
data.head()

,ENSG00000000419.14,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.17,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.15,ENSG00000001460.18,ENSG00000001461.17,...,ENSG00000289720.1,ENSG00000289721.1,ENSG00000289768.1,ENSG00000290146.1,ENSG00000290292.10,ENSG00000290318.1,ENSG00000291237.1,ENSG00000291309.1,ENSG00000291317.1,ENSG00000292277.1
Barcode,,,,,,,,,,,,,,,,,,,,,
42425466,-0.025602,-0.120398,0.045326,0.035264,0.910862,0.273816,0.391231,0.021851,0.725922,-0.079202,...,-0.224376,0.036207,-0.194905,0.007171,-0.025849,0.187269,0.028904,-1.731970,-0.107424,-0.114298
42425485,0.167509,-0.016713,-0.159610,-0.176183,-1.307548,0.069263,0.336547,0.112440,0.007640,-0.012886,...,-0.442563,-0.303264,0.167107,-0.438479,-0.016956,0.146238,0.361422,0.242226,-0.232664,0.352157
42425503,0.013347,-0.036949,-0.325952,0.285365,0.250657,-0.007391,-0.391654,-0.074956,0.226705,0.425440,...,-0.509482,-0.452852,0.137157,-0.281968,-0.021764,-0.424217,-0.124633,-1.724809,0.304143,-0.298029
42425507,0.504928,0.135288,-0.074574,0.263099,0.499940,0.175403,-0.082560,-0.095084,-0.034696,-0.147939,...,0.296849,-0.276431,-0.168831,-0.346269,0.297915,0.040784,-0.031955,-0.562990,-0.474290,0.390742
42425517,0.025635,0.080433,0.150633,-0.082066,-1.035806,0.219007,-0.163517,0.035354,0.227039,-0.057263,...,-0.347677,-0.507203,0.730585,0.230120,0.127151,-0.682757,-0.076163,0.369195,-0.087412,-0.064562


In [50]:
output_dir = "KNN_Networks/"
os.makedirs(output_dir, exist_ok=True)

In [52]:
data_values = data.T # Transpose to have genes as rows
genes = list(data_values.index)
print(genes)
data_values=data_values.values


['ENSG00000000419.14', 'ENSG00000000457.14', 'ENSG00000000460.17', 'ENSG00000000938.13', 'ENSG00000000971.17', 'ENSG00000001036.14', 'ENSG00000001084.13', 'ENSG00000001167.15', 'ENSG00000001460.18', 'ENSG00000001461.17', 'ENSG00000001561.7', 'ENSG00000001629.10', 'ENSG00000001630.18', 'ENSG00000001631.17', 'ENSG00000002016.18', 'ENSG00000002330.14', 'ENSG00000002549.13', 'ENSG00000002587.10', 'ENSG00000002726.21', 'ENSG00000002745.13', 'ENSG00000002822.16', 'ENSG00000002834.19', 'ENSG00000002919.15', 'ENSG00000002933.9', 'ENSG00000003056.8', 'ENSG00000003147.19', 'ENSG00000003249.15', 'ENSG00000003393.16', 'ENSG00000003400.16', 'ENSG00000003402.21', 'ENSG00000003436.16', 'ENSG00000003509.16', 'ENSG00000003756.17', 'ENSG00000003987.14', 'ENSG00000004059.11', 'ENSG00000004139.14', 'ENSG00000004142.12', 'ENSG00000004399.13', 'ENSG00000004455.18', 'ENSG00000004468.13', 'ENSG00000004478.8', 'ENSG00000004487.18', 'ENSG00000004534.15', 'ENSG00000004660.15', 'ENSG00000004700.16', 'ENSG00000004

In [53]:
#standardize the data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(data_values)
scaled_data=scaler.transform(data_values)



In [54]:


for k in range(4, 16):
    print(f"Generating KNN network for k={k}...")


    knnData = NNDescent(data_values,metric="correlation",n_neighbors=k,verbose=True).neighbor_graph

    edges = []
    weights = []
    neighborIndices = []
    for sourceIndex,theNeighbors in enumerate(knnData[0]):
        for neighborIndex,neighbor in enumerate(theNeighbors):
            if(sourceIndex==neighbor):
                continue
            similarity = np.mean(data_values[sourceIndex,:]*data_values[neighbor,:])
            edges.append((sourceIndex,neighbor))
            weights.append(similarity)
            neighborIndices.append(neighborIndex)



    g = ig.Graph(data_values.shape[0],edges=edges,edge_attrs={"weight":weights, "neighborIndex":neighborIndices})
    g = g.simplify(multiple=False,loops=True,combine_edges={"weight":"sum"})

    g.vs["Label"] = genes




    xn.igraph2xnet(g,output_dir+f"knn_{k}.xnet")
    #remove links with 0 or negative weights
    g.es.select(weight_lt=0).delete()
    xn.igraph2xnet(g,output_dir+f"knn_{k}_pos.xnet")


print("All networks generated and saved successfully.")

Generating KNN network for k=4...
Mon Feb  3 19:47:16 2025 Building RP forest with 16 trees
Mon Feb  3 19:47:16 2025 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	Stopping threshold met -- exiting after 6 iterations
Generating KNN network for k=5...
Mon Feb  3 19:47:17 2025 Building RP forest with 16 trees
Mon Feb  3 19:47:17 2025 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	 7  /  14
	Stopping threshold met -- exiting after 7 iterations
Generating KNN network for k=6...
Mon Feb  3 19:47:18 2025 Building RP forest with 16 trees
Mon Feb  3 19:47:18 2025 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	 7  /  14
	Stopping threshold met -- exiting after 7 iterations
Generating KNN network for k=7...
Mon Feb  3 19:47:19 2025 Building RP forest with 16 trees
Mon Feb  3 19:47:19 2025 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	